# Train YOLOv2 on LISA Dataset
-----------------------------------------------


In [1]:
import sys
print(sys.version) # Check Python Version

3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:09:58) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [2]:
import numpy as np
import os
from keras.optimizers import Adam
from utils.parse_input import load_data    # Data handler for LISA dataset
from yolov2.model import YOLOv2, darknet19
from utils.data_generator import flow_from_list

Using TensorFlow backend.


### Prepare LISA Dataset

In [3]:
lisa_path = "/home/dat/Downloads/training/" # Remember the `/` at the end
# pretrained_path = "/home/ubuntu/dataset/darknet19_544.weights"

In [4]:
x_train, y_train = load_data('training.txt')
labels           = np.unique(y_train[:,1])
num_classes      = len(labels)            # Count number of classes in the dataset

print("Train: {} samples\nNumber of classes: {}".format(len(x_train),num_classes))

Number of ground truth boxes: 3672 boxes
Train: 3672 samples
Number of classes: 31


### Construct YOLOv2 On Keras

In [6]:
darknet19 = darknet19(freeze_layers=True)
yolov2    = YOLOv2(feature_extractor=darknet19, num_anchors=5, num_classes=31)
model     = yolov2.model

# print(model.summary())
# HYPER-PARAMETERS
BATCH_SIZE = 64
EPOCHS     = 5
LEARN_RATE = 0.01
anchors    = np.array(((0.57273, 0.677385),
                     (1.87446, 2.06253),
                     (3.33843, 5.47434),
                     (7.88282, 3.52778),
                     (9.77052, 9.16828)))

In [24]:
import keras.backend as K
import tensorflow as tf
def yolo_loss(anchors, n_classes):

    def custom_loss(y_true, y_pred):

#         CONV_SHAPE = tf.cast(K.shape(y_pred)[1:3], dtype=tf.int32)

        GRID_W, GRID_H     = 40, 30
#         GRID_W, GRID_H = CONV_SHAPE[0], CONV_SHAPE[1]  # a tiny hack to get width, height of output layer
        NORM_W, NORM_H = GRID_W * 32, GRID_H * 32      # Scale back to get image input size

        SCALE_NOOB, SCALE_CONF, SCALE_COOR, SCALE_PROB = 0.5, 5.0, 5.0, 1.0
    
        y_pred = K.reshape(y_pred, [-1, GRID_W, GRID_H, len(anchors), n_classes + 5])
        # Adjust prediction  : bx = sigmoid(tx) + cx
        pred_box_xy = tf.sigmoid(y_pred[:, :, :, :, :2])

        # adjust w and h : bw = pw*exp(tw)
        pred_box_wh   = tf.exp(y_pred[:, :, :, :, 2:4]) * np.reshape(anchors, [1, 1, 1, len(anchors), 2])
        
        # scaling wh relative to top-left corner of feature map
        pred_box_wh   = tf.sqrt(pred_box_wh / np.reshape([float(GRID_W), float(GRID_H)], [1, 1, 1, 1, 2]))
        pred_box_conf = tf.expand_dims(tf.sigmoid(y_pred[:, :, :, :, 4]), -1)         # adjust confidence
        pred_box_prob = tf.nn.softmax(y_pred[:, :, :, :, 5:])          # adjust probability

        y_pred = tf.concat([pred_box_xy, pred_box_wh, pred_box_conf, pred_box_prob], 4)

        # Adjust ground truth
        # adjust x and y
        gt_dim    = K.shape(y_true)
        y_true    = K.reshape(y_true, [gt_dim[0], GRID_W, GRID_H, gt_dim[1], gt_dim[2]])
        center_xy = .5 * (y_true[:, :, :, :, 0:2] + y_true[:, :, :, :, 2:4])
        center_xy = center_xy / np.reshape([(float(NORM_W) / GRID_W), (float(NORM_H) / GRID_H)], [1, 1, 1, 1, 2])
        true_box_xy = center_xy - tf.floor(center_xy)

        # adjust w and h
        true_box_wh = (y_true[:, :, :, :, 2:4] - y_true[:, :, :, :, 0:2])
        true_box_wh = tf.sqrt(true_box_wh / np.reshape([float(NORM_W), float(NORM_H)], [1, 1, 1, 1, 2]))

        # adjust confidence
        pred_tem_wh   = tf.pow(pred_box_wh, 2) * np.reshape([GRID_W, GRID_H], [1, 1, 1, 1, 2])
        pred_box_area = pred_tem_wh[:, :, :, :, 0] * pred_tem_wh[:, :, :, :, 1]
        pred_box_ul   = pred_box_xy - 0.5 * pred_tem_wh
        pred_box_bd   = pred_box_xy + 0.5 * pred_tem_wh

        true_tem_wh   = tf.pow(true_box_wh, 2) * np.reshape([GRID_W, GRID_H], [1, 1, 1, 1, 2])
        true_box_area = true_tem_wh[:, :, :, :, 0] * true_tem_wh[:, :, :, :, 1]
        true_box_ul   = true_box_xy - 0.5 * true_tem_wh  # upper left
        true_box_br   = true_box_xy + 0.5 * true_tem_wh  # lower right

        # Find the IOU between GT and prediction
        intersect_ul   = tf.maximum(pred_box_ul, true_box_ul)
        intersect_br   = tf.minimum(pred_box_bd, true_box_br)
        intersect_wh   = intersect_br - intersect_ul
        intersect_wh   = tf.maximum(intersect_wh, 0.0)
        intersect_area = intersect_wh[:, :, :, :, 0] * intersect_wh[:, :, :, :, 1]

        iou           = tf.truediv(intersect_area, true_box_area + pred_box_area - intersect_area)
        best_box      = tf.equal(iou, tf.reduce_max(iou, [3], True))
        best_box      = tf.to_float(best_box)
        true_box_conf = tf.expand_dims(best_box * y_true[:, :, :, :, 4], -1)

        # adjust confidence
        true_box_prob = y_true[:, :, :, :, 5:]
        y_true        = tf.concat([true_box_xy, true_box_wh, true_box_conf, true_box_prob], 4)
        # y_true = tf.Print(y_true, [true_box_wh], message='DEBUG', summarize=30000)

        # ## Compute the weights
        weight_coor = tf.concat(4 * [true_box_conf], 4)
        weight_coor = SCALE_COOR * weight_coor
        weight_conf = SCALE_NOOB * (1. - true_box_conf) + SCALE_CONF * true_box_conf
        weight_prob = tf.concat(n_classes* [true_box_conf], 4)
        weight_prob = SCALE_PROB * weight_prob
        weight = tf.concat([weight_coor, weight_conf, weight_prob], 4)

        # ## Finalize the loss
        loss = tf.pow(y_pred - y_true, 2)
        loss = loss * weight
        loss = tf.reshape(loss, [-1, GRID_W * GRID_H * len(anchors) * (4 + 1 + n_classes)])
        loss = tf.reduce_sum(loss, 1)
        loss = .5 * tf.reduce_mean(loss)

        return loss

    return custom_loss

In [25]:
# Compile and train model
# from yolov2.loss import yolo_loss

train_data_gen = flow_from_list(x_train, y_train, anchors, batch_size=BATCH_SIZE)

model.compile(optimizer= Adam(LEARN_RATE),
              loss     = yolo_loss(anchors, num_classes))

In [26]:
hist =  model.fit_generator(generator       = train_data_gen, 
                            steps_per_epoch = len(x_train) / BATCH_SIZE, 
                            epochs          = 10, 
                            workers=1, 
                            pickle_safe=False, 
                            verbose=1, 
                            initial_epoch   = 0,
                            callbacks=[tf_board, check_pt])
model.save_weights('darknet_weights.h5')

NameError: name 'tf_board' is not defined

## Multi-GPUs Training - Data Parallelism Approach

* Each GPU will have a copy of the model
* During training time, mean of all gradidents from each GPU will be calculated to update the model
<img style="width:40%" src="https://www.tensorflow.org/images/Parallelism.png">

### Visualize training process using Tensorboard
Open `http://<public-dns>:6006`

In [ ]:
import cv2
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

def extract_sign(img, bbox, output_size=(32, 32)):
    xc, yc, w, h = bbox.x, bbox.y, bbox.w, bbox.h
    x1   = int(xc - w/2)
    y1   = int(yc - h/2)
    x2   = int(xc + w/2)
    y2   = int(yc + h/2)
    roi = img[y1:y2, x1:x2]
    roi = cv2.resize(roi, output_size)
    return roi
x_train, y_train = shuffle(x_train, y_train)
fig = plt.figure(figsize=(17, 8))
for i, label in enumerate(labels):
    ax           = fig.add_subplot(4, 8, 1 + i, xticks=[], yticks=[])
    idx          = np.where(y_train[:, 1] == label)[0][0]
    img          = cv2.cvtColor(cv2.imread(x_train[idx]), cv2.COLOR_BGR2RGB)
    box          = y_train[idx][0]
    sign_only    = extract_sign(img, box, (32, 32)) #  just extract the sign
    ax.set_title(label)
    plt.imshow(sign_only)
plt.show()

In [ ]:
## Count frequencies of each traffic sign
labels, frequencies = np.unique(y_train[:,1 ], return_counts=True)

plt.figure(figsize=(20, 5))
x = np.arange(len(labels))
plt.xticks(x, labels, rotation=70)
plt.bar(x, frequencies)